In [ ]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/NNFL-Assignment2/data5.xlsx', header=None)
df.head()

,0,1,2,3,4,5,6,7
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [ ]:
df = df.drop_duplicates(keep='first')
df = df.sample(frac = 1, random_state = 7).reset_index(drop=True)
m = df.shape[0]
n = df.shape[1] - 1
print(m,n)
df.head()

210 7


,0,1,2,3,4,5,6,7
0,18.94,16.49,0.8750,6.445,3.639,5.0640,6.362,2
1,12.88,13.50,0.8879,5.139,3.119,2.3520,4.607,1
2,18.36,16.52,0.8452,6.666,3.485,4.9330,6.448,2
3,15.88,14.90,0.8988,5.618,3.507,0.7651,5.091,1
4,13.22,13.84,0.8680,5.395,3.070,4.1570,5.088,1


In [ ]:
datan = df.values
X = datan[:, 0:7]

X = (X - np.min(X, axis = 0))/(np.max(X, axis=0)-np.min(X, axis=0))
X = np.concatenate((np.ones((m,1)),X), axis=1)
#X = (X - X.mean(axis = 0))/(X.std(axis = 0))
y = datan[:, -1:]
print(X.shape, y.shape)

(210, 8) (210, 1)


In [ ]:
num_classes = len(np.unique(y))
print(num_classes)

3


In [ ]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

(151, 8) (17, 8) (42, 8) (151, 1) (17, 1) (42, 1)


In [ ]:
def one_hot(y):
  len = np.size(y)
  encoded_y = np.zeros((len,3))
  for i in range(len):
    if y[i] == 1:
      encoded_y[i,0] = 1
    if y[i] == 2:
      encoded_y[i,1] = 1
    if y[i] == 3:
      encoded_y[i,2] = 1
  return encoded_y

In [ ]:
y_train_encoded = one_hot(y_train)
y_val_encoded = one_hot(y_val)
y_test_encoded = one_hot(y_test)

In [ ]:
def sigmoid(x,w):
  z = np.dot(x,w.T)
  return 1/(1 + np.exp(-z))

In [ ]:
def auto_encoder(X,alpha,lm,iters):
  w1 = np.random.rand(X.shape[1]-1,X.shape[1])
  w2 = np.random.rand(X.shape[1],X.shape[1]-1)
  
  for i in range(iters):
    h1 = sigmoid(X,w1)
    h2 = sigmoid(h1,w2)
    del2 = (h2-X)*h2*(1-h2)
    del1 = np.dot(del2,w2)*h1*(1-h1)
    w2 = w2 - alpha*(np.dot(del2.T,h1) + lm*w2)
    w1 = w1 - alpha*(np.dot(del1.T,X) + lm*w1)
    
  return w1,sigmoid(X,w1)

In [ ]:
def stacked_auto_encoder(X, alpha,lm,iters):
  w1,h1 = auto_encoder(X_train,0.06,0.00001,10000)
  w2,h2 = auto_encoder(h1,0.06,0.00001,10000)
  w3,h3 = auto_encoder(h2,0.03,0.00001,10000)


  w4 = np.random.rand(h3.shape[1]-2,h3.shape[1])
  alpha1 = 100*alpha
  for i in range(iters):
    h1=sigmoid(X_train,w1)
    h2=sigmoid(h1,w2)
    h3=sigmoid(h2,w3)
    h4 = sigmoid(h3,w4)
    del4 = (h4-y_train_encoded)*h4*(1-h4)
    del3 = np.dot(del4,w4)*h3*(1-h3)
    del2 = np.dot(del3,w3)*h2*(1-h2)
    del1 = np.dot(del2,w2)*h1*(1-h1)
    w4 = w4 - (100*alpha)*(np.dot(del4.T,h3) + lm*w4)
    w3 -= alpha*(np.dot(del3.T,h2) + lm*w3)
    w2 -= alpha*(np.dot(del2.T,h1) + lm*w2)
    w1 -= alpha*(np.dot(del1.T,X_train) + lm*w1)
    
  y = sigmoid(sigmoid(sigmoid(sigmoid(X,w1),w2),w3),w4)
  return y

In [ ]:
def confusion_matrix(predictions, labels):
  conf_matrix = [0,0,0,0,0,0,0,0,0]
  m = labels.shape[0]
  for i in range(m):
    conf_matrix[3*(labels[i][0]-1) + predictions[i] - 1] = conf_matrix[3*(labels[i][0]-1) + predictions[i] - 1] + 1 
  return conf_matrix

In [ ]:
alpha_vals = np.linspace(0.0001, 0.01, 5).tolist()
lm_vals = np.linspace(0.01, 0.1, 5).tolist()
acc_best = 0
alpha_best, lm_best = 0, 0
for i in alpha_vals:
  for j in lm_vals:
    ypred_val = stacked_auto_encoder(X_val, i, j, 500)
    ypred_val = (np.argmax(ypred_val, axis=1) + 1)
    conf_matrix_val = confusion_matrix(ypred_val.astype(int), y_val.astype(int))
    acc = (conf_matrix_val[0] + conf_matrix_val[4] + conf_matrix_val[8])/sum(conf_matrix_val)
    if acc > acc_best:
      acc_best = acc
      lm_best = j
      alpha_best = i

ypred = stacked_auto_encoder(X_test, alpha_best,lm_best,500)

In [ ]:
ypred = (np.argmax(ypred, axis=1) + 1)
ypred

array([2, 1, 2, 2, 1, 1, 3, 3, 3, 3, 1, 2, 3, 3, 3, 2, 3, 2, 1, 3, 3, 3,
       3, 1, 1, 2, 2, 3, 3, 2, 2, 1, 2, 1, 1, 2, 2, 2, 3, 2, 3, 1])

In [ ]:
conf_matrix = confusion_matrix(ypred.astype(int),y_test.astype(int))

ind_acc1=conf_matrix[0]/sum(conf_matrix[0:3])
ind_acc2=conf_matrix[4]/sum(conf_matrix[3:6])
ind_acc3=conf_matrix[8]/sum(conf_matrix[6:9]) 
overall=(conf_matrix[0] + conf_matrix[4] + conf_matrix[8])/sum(conf_matrix)

In [ ]:
print("Individual accuracy 1: ",ind_acc1)
print("Individual accuracy 2: ",ind_acc2)
print("Individual accuracy 3: ",ind_acc3)
print("Overall accuracy : ", overall)

Individual accuracy 1:  0.8333333333333334
Individual accuracy 2:  0.9285714285714286
Individual accuracy 3:  1.0
Overall accuracy :  0.9285714285714286
